In [1]:
import pandas as pd
import requests
import json

In [4]:
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    granularity = 60 # 86400
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity={granularity}'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            return data
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

In [5]:
pair = "BTC/USD"
data = fetch_daily_data(symbol=pair)

In [7]:
data

,unix,low,high,open,close,volume,date,vol_fiat
0,1613328240,48498.99,48599.33,48499.68,48549.52,15.251919,2021-02-14 18:44:00,7.404733e+05
1,1613328180,48480.01,48527.76,48527.76,48501.00,15.152758,2021-02-14 18:43:00,7.349239e+05
2,1613328120,48500.00,48543.72,48510.00,48527.76,27.730931,2021-02-14 18:42:00,1.345720e+06
3,1613328060,48509.71,48552.29,48511.90,48510.00,3.879654,2021-02-14 18:41:00,1.882020e+05
4,1613328000,48501.10,48525.46,48509.48,48511.89,11.347935,2021-02-14 18:40:00,5.505098e+05
...,...,...,...,...,...,...,...,...
295,1613310540,48713.50,48915.92,48915.92,48870.94,55.275528,2021-02-14 13:49:00,2.701367e+06
296,1613310480,48825.00,49019.95,49013.30,48915.22,30.825376,2021-02-14 13:48:00,1.507830e+06
297,1613310420,49011.56,49133.78,49129.54,49015.39,13.681719,2021-02-14 13:47:00,6.706148e+05
298,1613310360,49100.00,49135.56,49118.26,49129.68,4.244815,2021-02-14 13:46:00,2.085464e+05


In [12]:

# IMPORTS
import pandas as pd
import math
import os.path
import time
#from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
#bitmex_api_key = '[REDACTED]'    #Enter your own API-key here
#bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = 'O6wf2bZomPq7qM4XnF86WpAIGNQPtTh5LOcNrkhc3TlqqirbI6Nl8yfoevVTpVhN'
binance_api_secret = '47NDuVw5RxS15UmzBCk8XGGxlF5oLE8hwVsumXklr8AmTa8rfIQ2jEULT2Kzy5QF'

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
#bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    #elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    #if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    
    # salva o appendi
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df
#❤

In [23]:
symbol = 'BTCUSDT'
data = get_all_binance(symbol, kline_size='1m', save = True)

All caught up..!


In [24]:
data.shape

(1832408, 11)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1832408 entries, 2017-08-17 04:00:00 to 2021-02-14 22:14:00
Data columns (total 11 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   open         object
 1   high         object
 2   low          object
 3   close        object
 4   volume       object
 5   close_time   int64 
 6   quote_av     object
 7   trades       int64 
 8   tb_base_av   object
 9   tb_quote_av  object
 10  ignore       object
dtypes: int64(2), object(9)
memory usage: 167.8+ MB


In [26]:
data

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-14 22:10:00,48868.04000000,48868.05000000,48857.90000000,48860.00000000,11.48072200,1613340659999,560979.03612802,887,4.47741900,218776.54263944,0
2021-02-14 22:11:00,48860.00000000,48884.71000000,48860.00000000,48867.67000000,31.36170900,1613340719999,1532538.23049018,1320,21.21716000,1036777.91942936,0
2021-02-14 22:12:00,48867.67000000,48897.03000000,48867.67000000,48891.64000000,23.70975100,1613340779999,1159092.87499973,1050,11.31830900,553302.83552121,0
